# Quantize Transform
This notebook illustrates the effects of quantizing signals. Different quantization methods have particular nonlinear effects on signal properties such as dynamic range.

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from torchsig.signals.signal_types import Signal
import torchsig.transforms.functional as F
from torchsig.utils.defaults import default_dataset

Generator function for test modulated signal input

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

    Args:
    num_iq_samples (int, optional): Length of sample. Defaults to 128.
    scale (float, optional): scale normalized signal data. Defaults to 1.0.

    Returns:
        signal: generated Signal.

    """
    dataset = default_dataset(
        signal_generators=["qpsk"],
        num_signals_min=1,
        num_signals_max=1,
        signal_duration_in_samples_min=num_iq_samples,
        signal_duration_in_samples_max=num_iq_samples,
        noise_power_db=0.0,
        start_in_samples=0,
        seed=42,
    )
    signal = dataset.signal_generators[0]()

    # normalize, then scale data
    signal.data = F.normalize(data=signal.data, norm_order=2, flatten=False)
    signal.data = signal.data * scale

    return signal

Create input test data at full scale complex64 precision.

In [ ]:
# test data
N = 1024
qpsk_data = generate_qpsk_signal(num_iq_samples=N, scale=1.0).data

# add in a tiny amount of noise to avoid log10(0)
complex_noise = np.sqrt(1e-6) * (
    np.random.normal(0, 1, N) + 1j * np.random.normal(0, 1, N)
)
qpsk_data += complex_noise

Illustrate the effects of 8-bit quantization for three different quantizing configurations

In [ ]:
# quantization: fixed configuration parameters
num_bits = 8
rounding_mode = "floor"


# simulate quantization at full scale
qpsk_data_full_scale = F.quantize(
    data=qpsk_data,
    num_bits=num_bits,
    ref_level_adjustment_db=0,
    rounding_mode=rounding_mode,
)

full_scale_sfdr = np.mean(np.abs(qpsk_data) ** 2) / np.mean(
    np.abs(qpsk_data - qpsk_data_full_scale) ** 2
)
full_scale_sfdr_db = 10 * np.log10(full_scale_sfdr)
enob_full_scale = full_scale_sfdr_db / 6.02

# simulate saturated ADC
qpsk_data_saturated = F.quantize(
    data=qpsk_data,
    num_bits=num_bits,
    ref_level_adjustment_db=5,
    rounding_mode=rounding_mode,
)

saturated_sfdr = np.mean(np.abs(qpsk_data) ** 2) / np.mean(
    np.abs(qpsk_data - qpsk_data_saturated) ** 2
)
saturated_sfdr_db = 10 * np.log10(saturated_sfdr)
enob_saturated = saturated_sfdr_db / 6.02

# loss in dynamic range
below_level_db = -35
qpsk_data_below_full_scale = F.quantize(
    data=qpsk_data,
    num_bits=num_bits,
    ref_level_adjustment_db=below_level_db,
    rounding_mode=rounding_mode,
)

below_full_scale_sfdr = np.mean(np.abs(qpsk_data) ** 2) / np.mean(
    np.abs(qpsk_data - qpsk_data_below_full_scale) ** 2
)
below_full_scale_sfdr_db = 10 * np.log10(below_full_scale_sfdr)
enob_below_full_scale = below_full_scale_sfdr_db / 6.02

Plot the Full Scale, Saturated, and below Full Scale effects

In [ ]:
fig = plt.figure(figsize=(16, 12))
fig.subplots_adjust(hspace=0.5)

ax = fig.add_subplot(3, 1, 1)
ax.plot(np.real(qpsk_data), label="Input")
ax.plot(np.real(qpsk_data_full_scale), label="Output")
ax.grid()
ax.set_title(
    f"Full Scale Quantization. SFDR = {full_scale_sfdr_db:0.1f} dB, ENOB = {enob_full_scale:0.1f} bits"
)
ax.set_xlabel("Time Index $n$", fontsize="large")
ax.set_ylabel("Amplitude")
ax.legend(fontsize="large", loc="upper left")

ax = fig.add_subplot(3, 1, 2)
ax.plot(np.real(qpsk_data), label="Input")
ax.plot(np.real(qpsk_data_saturated), label="Saturated Quantization")
ax.grid()
ax.set_title(
    f"Quantization when Saturated. SFDR = {saturated_sfdr_db:0.1f} dB, ENOB = {enob_saturated:0.1f} bits"
)
ax.set_xlabel("Time Index $n$", fontsize="large")
ax.set_ylabel("Amplitude")

ax = fig.add_subplot(3, 1, 3)
ax.plot(np.real(qpsk_data), label="Input")
ax.plot(np.real(qpsk_data_below_full_scale), label="Dynamic-Range Loss Quantization")
ax.grid()
ax.set_title(
    f"Quantization when Below Full Scale by {below_level_db:0.1f} dB. SFDR = {below_full_scale_sfdr_db:0.1f}, ENOB = {enob_below_full_scale:0.1f} bits"
)
ax.set_xlabel("Time Index $n$", fontsize="large")
ax.set_ylabel("Amplitude")